In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [196]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 
import random

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.


# Constants from lab 4b
root2 = math.sqrt(2)

denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 


W_transform_matrix = [ [ -1 / denom1 , (1 + root2) / denom1 ],
                        [  1 / denom2 , (root2 - 1) / denom2 ] ]
W_transform = Operator(W_transform_matrix) 


V_transform_matrix = [ [  1 / denom1 , (1 + root2) / denom1 ],
                        [ -1 / denom2 , (root2 - 1) / denom2 ] ]
V_transform = Operator(V_transform_matrix) 

# ==== Step 1: Alice and Bob each receive one qubit of an entangled pair in state 1/√2 (|01⟩−|10⟩). ====
def create_entangled_pair():
    q = QuantumCircuit(2) 
    q.h(0)
    q.cx(0,1)
    return q

# ==== Step 2/3: Alice/Bob chooses an operator Ai/Bj randomly from her/his set of three, with probability 1/3 each. ====
def one_third_probability_circuit():
    # Create the qubit in state 1/root(3) |0> + root(2)/root(3) |1>
    # To do this I need a unitary operator which can take a qubit in state |0> (= [1 0].T]) and transform it into the required state
    # The unitary operator, U, should have column 1 be [1/root(3) root(2)/root(3)] since this will multiply with 1 to give the required state
    # In order to be unitary we need each column to be orthogonal and to have unit norm
    # Clearly [1/root(3) root(2)/root(3)] has unit norm
    # It remains to construct a second column [x y] which is orthogonal and has unit norm
    # We can introduce the following system of equations :
    # Unit norm: x^2 + y^2 = 1                         (1)
    # Orthogonal: 1/root(3)*x + root(2)/root(3)*y = 0  (2)
    # Rearange (2) for x gives: x = -root(2)*y
    # Substitute x into (1) gives: (-root(2)*y)^2 + y^2 = 1  => 2y^2 + y^2 = 1  => y = 1/root(3)
    # Substitue y into (2) gives: x = -root(2)*(1/root(3)) = -root(2)/root(3)
    # Therefore the unitary operator to produce the required state is:
    # U = [ 1/root(3)          -root(2)/root(3)
    #       root(2)/root(3)    1/root(3)       ]

    # Construct U
    U = [ [ 1/math.sqrt(3) , -math.sqrt(2)/math.sqrt(3) ],
          [ math.sqrt(2)/math.sqrt(3 ), 1/math.sqrt(3)  ] ]
    U_gate = Operator(U)

    qc = QuantumCircuit(1)

    # Apply the custom unitary matrix to the qubit
    qc.unitary(U_gate, [0])
    
    qc.measure_all()  # Measure the qubit
    
    # Simulate using BasicSimulator
    backend = BasicSimulator()
    compiled = transpile(qc, backend)
    job_sim = backend.run(compiled, shots=1)  # Run many shots to estimate probabilities
    result_sim = job_sim.result()
    counts = result_sim.get_counts(compiled)
    return counts
    
# Choose randomly between the 3 operators
def random_choice():

    random_0_1 = one_third_probability_circuit()
    # random_0_1 will be a 0 with a 1/3 chance
    # If we got a 0 then that is the random number we got with probability 1/3
    if "0" in random_0_1:
        return 0
    else: # Else, if we got 1 (with probability 2/3) then we do a 50/50 chance between 1 and 2
        circuit = QuantumCircuit(1, 1)
        circuit.h(0)
        circuit.measure(0, 0)
        backend = BasicSimulator()
        compiled = transpile(circuit, backend)
        job_sim = backend.run(compiled, shots=1)
        second_result = job_sim.result().get_counts()
        return 1 if "0" in second_result else 2


In [197]:
# This is just to check that the random number 1-3 works properly
from collections import Counter

choices = []
for i in range(1000):
    choices.append(random_choice())

counts = Counter(choices)
print(counts)

Counter({1: 348, 2: 332, 0: 320})


In [198]:
# From Lab 4B
def average(c,n): 
    backend = BasicSimulator()
    compiled = transpile(c, backend)
    job_sim = backend.run(compiled, shots=n)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)
    #print(counts)
    count00 = counts.get("00",0) 
    count01 = counts.get("01",0) 
    count10 = counts.get("10",0) 
    count11 = counts.get("11",0) 
    # The return value includes the conversion from measurement results 0,1 to +1,-1
    # Each 00 means a value of  1 (+1 * +1)
    # Each 01 means a value of -1 (+1 * -1)
    # Each 10 means a value of -1 (-1 * +1)
    # Each 11 means a value of  1 (+1 * +1)
    return (count00 - count01 - count10 + count11) / n 


def ekert91_protocol(N):
    # Alice's and Bob's measurement operators
    alice_operators = ['X', 'W', 'Z']  # A1, A2, A3
    bob_operators = ['W', 'Z', 'V']    # B1, B2, B3
    
    alice_bases = []
    bob_bases = []

    # Store measurements for Bell inequality calculation
    alice_X_bob_W = []  # Measurements of XxW
    alice_X_bob_V = []  # Measurements of XxV
    alice_Z_bob_W = []  # Measurements of ZxW
    alice_Z_bob_V = []  # Measurements of ZxV

    # Shared key
    shared_key = []

    # Process each entangled pair
    for i in range(math.ceil((9 * N) / 2)):
        # 1: Alice and Bob each receive one qubit of an entangled pair in state 1/√2 (|01⟩−|10⟩)
        qc = create_entangled_pair()
        
        # 2: Alice chooses an operator Ai randomly from her set of three, with probability 1/3 each
        alice_choice = random_choice()
        alice_operator = alice_operators[alice_choice]
        alice_bases.append(alice_choice)

        # 3: Bob chooses an operator Bi randomly from his set of three, with probability 1/3 each
        bob_choice = random_choice()
        bob_operator = bob_operators[bob_choice]
        bob_bases.append(bob_choice)

        # 4: Alice measures operator Ai on her qubit
        if alice_operator == 'X':
            qc.h(0)
        elif alice_operator == 'W':
            qc.unitary(W_transform_matrix, [0]) # Apply the transform (so when we measure it in the standard basis it is as if we measured in the W basis, i think?)
        # For Z, we do nothing (standard basis)
        
        # 5: Bob measures operator Bj on hist qubit
        if bob_operator == 'W':
            qc.unitary(W_transform_matrix, [1], label='W_Bob') # Apply the transform (W basis)
        elif bob_operator == 'V':
            qc.unitary(V_transform_matrix, [1], label='V_Bob') # Apply the transform (V basis)
        # For Z, we do nothing (standard basis)
        
        # Now actually do the measurement
        qc.measure_all()
        Ai_Bi_average = average(qc,100) 

        # Collect measurements for Bell inequality calculation
        if alice_choice == 0 and bob_choice == 0:  # A1 (X) and B1 (W)
            alice_X_bob_W.append((Ai_Bi_average))
        elif alice_choice == 0 and bob_choice == 2:  # A1 (X) and B3 (V)
            alice_X_bob_V.append((Ai_Bi_average))
        elif alice_choice == 2 and bob_choice == 0:  # A3 (Z) and B1 (W)
            alice_Z_bob_W.append((Ai_Bi_average))
        elif alice_choice == 2 and bob_choice == 2:  # A3 (Z) and B3 (V)
            alice_Z_bob_V.append((Ai_Bi_average))
        # This covers 4/9 of the combinations 
        # We discard the other 3 combinations ((X,Z), (W,Z), and (W, V)) - This covers another 3/9 of the combinations (7/9 total)


        # Get Alice and Bob bits
        backend = BasicSimulator()
        compiled = transpile(qc, backend)
        job_sim = backend.run(compiled, shots=1)
        result = job_sim.result().get_counts(compiled)
        outcome = list(result.keys())[0]
        
        alice_bit = int(outcome[0])
        bob_bit = int(outcome[1])
        
        # Check if bases match for key generation
        # A2 (W) and B1 (W) are the same basis
        # A3 (Z) and B2 (Z) are the same basis
        # This covers the remaining 2/9 of the combinations
        # if            i = 2 and j = 1             or             i = 3 and j = 2
        if (alice_choice == 1 and bob_choice == 0) or (alice_choice == 2 and bob_choice == 1):
            shared_key.append(alice_bit)  # Bob's result is automatically opposite / could append inverted bob_result here instead

    # Now we have built the shared key and collected the frequencies needed to calculaate S
    
    # Calculate S value for Bell inequality
    # S = |⟨X ⊗ W⟩ − ⟨X ⊗ V ⟩ + ⟨Z ⊗ W⟩ + ⟨Z ⊗ V ⟩|.
    X_W_avg = sum(alice_X_bob_W) / len(alice_X_bob_W) if alice_X_bob_W else 0
    X_V_avg = sum(alice_X_bob_V) / len(alice_X_bob_V) if alice_X_bob_V else 0
    Z_W_avg = sum(alice_Z_bob_W) / len(alice_Z_bob_W) if alice_Z_bob_W else 0
    Z_V_avg = sum(alice_Z_bob_V) / len(alice_Z_bob_V) if alice_Z_bob_V else 0

    S = abs(X_W_avg - X_V_avg + Z_W_avg + Z_V_avg)
    
    return {
        'shared_key': shared_key,
        'key_length': len(shared_key),
        'bell_value': S,
        'alice_bases': alice_bases,
        'bob_bases': bob_bases,
    }

# Run the protocol
N = 20
print(f"Running Ekert91 protocol to establish a {N}-bit key...")
results = ekert91_protocol(N)

print("\n=== Ekert91 Protocol Results ===")
print(f"Shared key: {results['shared_key']}")
print(f"Key length: {results['key_length']}")
print(f"Bell value |S|: {results['bell_value']:.4f}")

Running Ekert91 protocol to establish a 20-bit key...

=== Ekert91 Protocol Results ===
Shared key: [1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1]
Key length: 20
Bell value |S|: 2.8328


In [199]:
# Run 10 times to check for errors
for i in range(10):
    results = ekert91_protocol(N)
    print("\n=== Ekert91 Protocol Results ===")
    print(f"Shared key: {results['shared_key']}")
    print(f"Key length: {results['key_length']}")
    print(f"Bell value |S|: {results['bell_value']:.4f}")


=== Ekert91 Protocol Results ===
Shared key: [1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
Key length: 20
Bell value |S|: 2.8152

=== Ekert91 Protocol Results ===
Shared key: [1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0]
Key length: 27
Bell value |S|: 2.8293

=== Ekert91 Protocol Results ===
Shared key: [1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0]
Key length: 27
Bell value |S|: 2.9308

=== Ekert91 Protocol Results ===
Shared key: [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
Key length: 16
Bell value |S|: 2.8589

=== Ekert91 Protocol Results ===
Shared key: [0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Key length: 13
Bell value |S|: 2.8914

=== Ekert91 Protocol Results ===
Shared key: [0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0]
Key length: 17
Bell value |S|: 2.8161

=== Ekert91 Protocol Results ===
Shared key: [0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0]
Key length: 17
Bell value |S|: 2